In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Bladder_Cancer"
cohort = "GSE201395"

# Input paths
in_trait_dir = "../../input/GEO/Bladder_Cancer"
in_cohort_dir = "../../input/GEO/Bladder_Cancer/GSE201395"

# Output paths
out_data_file = "../../output/preprocess/Bladder_Cancer/GSE201395.csv"
out_gene_data_file = "../../output/preprocess/Bladder_Cancer/gene_data/GSE201395.csv"
out_clinical_data_file = "../../output/preprocess/Bladder_Cancer/clinical_data/GSE201395.csv"
json_path = "../../output/preprocess/Bladder_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"An in vitro study of the development of resistance to FGFR inhibition in two urothelial carcinoma cell lines"
!Series_summary	"Expression array data was used to compare parental FGFR3-TACC3 fusion-driven urothelial cell lines with their FGFR inhibitor-resistant derivatives."
!Series_summary	"In this dataset, we include RT112 and RT4 parental cells, RT112 cells acutely treated with PD173074 (24 h), RT112 and RT4 resistant derivatives cultured with drug and their resistant derivatives cultured for four to six passages out of drug."
!Series_overall_design	"11 cell lines were analysed on the Affymetrix HTA 2.0 platform: RT112 and RT4 parental cells (RT112 parental no PD; RT4 parental no PD), RT112 cells acutely treated with PD173074 (24 h) (RT112 parental PD), RT112 and RT4 resistant derivatives cultured with drug (RT112 R1 PD; RT112 R2 PD; RT112 R3 PD; RT4 R1 PD) and their resistant derivatives cultured for four to six passages out of drug (RT112 R1 

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the Series overall design, this dataset uses Affymetrix HTA 2.0 platform
# which is a gene expression microarray, so gene expression data should be available
is_gene_available = True

# 2.1 Data Availability
# From the sample characteristics dictionary, we can see that this dataset contains information
# about cell lines, not human patients. The dataset is comparing parental cell lines with
# resistant derivatives, which is not directly related to our trait (Bladder_Cancer)
# There is no information about age or gender since these are cell lines
trait_row = 0  # The cell line information can be used to infer bladder cancer status
age_row = None  # No age information available for cell lines
gender_row = None  # No gender information available for cell lines

# 2.2 Data Type Conversion
def convert_trait(value):
    """Convert cell line information to binary trait (Bladder_Cancer)."""
    if value is None:
        return None
    
    # Extract value after colon if present
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # All cell lines in this dataset are urothelial carcinoma (bladder cancer) cell lines
    # So they all would be classified as having the trait
    if "urothelial carcinoma" in value.lower():
        return 1
    else:
        return None

# No age conversion function needed as age data is not available
def convert_age(value):
    return None

# No gender conversion function needed as gender data is not available
def convert_gender(value):
    return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None

# Validate and save cohort info
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # We need to check if clinical_data is already defined from a previous step
    try:
        clinical_data
    except NameError:
        # If clinical_data is not defined, we need to notify about this issue
        print("Warning: clinical_data DataFrame is not available from previous steps.")
    else:
        # Extract clinical features
        clinical_df = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview the clinical dataframe
        preview = preview_df(clinical_df)
        print("Clinical Data Preview:")
        print(preview)
        
        # Create directory if it doesn't exist
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        
        # Save the clinical data
        clinical_df.to_csv(out_clinical_data_file)
        print(f"Clinical data saved to {out_clinical_data_file}")


Clinical Data Preview:
{'GSM6062606': [1.0], 'GSM6062607': [1.0], 'GSM6062608': [1.0], 'GSM6062609': [1.0], 'GSM6062610': [1.0], 'GSM6062611': [1.0], 'GSM6062612': [1.0], 'GSM6062613': [1.0], 'GSM6062614': [1.0], 'GSM6062615': [1.0], 'GSM6062616': [1.0], 'GSM6062617': [1.0], 'GSM6062618': [1.0], 'GSM6062619': [1.0], 'GSM6062620': [1.0], 'GSM6062621': [1.0], 'GSM6062622': [1.0], 'GSM6062623': [1.0], 'GSM6062624': [1.0], 'GSM6062625': [1.0], 'GSM6062626': [1.0], 'GSM6062627': [1.0], 'GSM6062628': [1.0], 'GSM6062629': [1.0], 'GSM6062630': [1.0], 'GSM6062631': [1.0], 'GSM6062632': [1.0], 'GSM6062633': [1.0], 'GSM6062634': [1.0], 'GSM6062635': [1.0], 'GSM6062636': [1.0], 'GSM6062637': [1.0], 'GSM6062638': [1.0]}
Clinical data saved to ../../output/preprocess/Bladder_Cancer/clinical_data/GSE201395.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row IDs (gene or probe identifiers) for future observation.
print(gene_data.index[:20])


Index(['2824546_st', '2824549_st', '2824551_st', '2824554_st', '2827992_st',
       '2827995_st', '2827996_st', '2828010_st', '2828012_st', '2835442_st',
       '2835447_st', '2835453_st', '2835456_st', '2835459_st', '2835461_st',
       '2839509_st', '2839511_st', '2839513_st', '2839515_st', '2839517_st'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# These identifiers appear to be probe IDs from an Affymetrix microarray, not human gene symbols
# They follow the format of numerical IDs with "_st" suffix, which is typical for Affymetrix arrays
# They need to be mapped to standard gene symbols for proper biological interpretation

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['TC01000001.hg.1', 'TC01000002.hg.1', 'TC01000003.hg.1', 'TC01000004.hg.1', 'TC01000005.hg.1'], 'probeset_id': ['TC01000001.hg.1', 'TC01000002.hg.1', 'TC01000003.hg.1', 'TC01000004.hg.1', 'TC01000005.hg.1'], 'seqname': ['chr1', 'chr1', 'chr1', 'chr1', 'chr1'], 'strand': ['+', '+', '+', '+', '+'], 'start': ['11869', '29554', '69091', '160446', '317811'], 'stop': ['14409', '31109', '70008', '161525', '328581'], 'total_probes': [49.0, 60.0, 30.0, 30.0, 191.0], 'gene_assignment': ['NR_046018 // DDX11L1 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 1 // 1p36.33 // 100287102 /// ENST00000456328 // DDX11L5 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 5 // 9p24.3 // 100287596 /// ENST00000456328 // DDX11L1 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 1 // 1p36.33 // 100287102', 'ENST00000408384 // MIR1302-11 // microRNA 1302-11 // --- // 100422919 /// ENST00000408384 // MIR1302-10 // microRNA 1302-10 // --- // 100422834 /// ENST0000040838

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Observe gene identifiers in gene data and gene annotation data
# In gene_data, identifiers have format like '2824546_st'
# In gene_annotation, the most similar identifiers would be 'probeset_id' but they are different format
# The 'gene_assignment' column contains gene symbols

# Let's get a better look at the 'ID' column in gene_data and check if it matches any column in gene_annotation
print("Gene data index preview:")
print(gene_data.index[:5])

# Let's try to find a matching ID column in the gene annotation data
# First check if any annotation columns contain matching formats
for col in gene_annotation.columns:
    if col in ['ID', 'probeset_id']:
        print(f"\nChecking {col} in gene_annotation:")
        print(gene_annotation[col][:5])

# It appears there's a mismatch between the probe IDs in gene_data and gene_annotation
# We need to determine how they relate to map them properly

# Looking at the preview, we need to see if our gene expression probes map to the annotation data
# Let's try to find a match by dropping the "_st" suffix

# 2. Create a mapping from probe IDs to gene symbols
# The 'gene_assignment' column contains gene information
# We need to extract gene symbols from this column

# Create a mapping dataframe by selecting the ID and gene_assignment columns
mapping_df = gene_annotation[['ID', 'gene_assignment']].copy()
mapping_df = mapping_df.rename(columns={'gene_assignment': 'Gene'})

# Print a sample of mapping_df to confirm it has the right structure
print("\nMapping dataframe sample:")
print(mapping_df.head(2))

# 3. Apply gene mapping to convert probe-level measurements to gene-level expression
# Use the apply_gene_mapping function from the library

# First, check if we can find any matches between gene_data and our mapping dataframe
print("\nFirst few probe IDs in gene expression data:")
print(gene_data.index[:5])
print("\nFirst few probe IDs in mapping dataframe:")
print(mapping_df['ID'][:5])

# It seems there might be format differences between the two datasets
# Let's try to find a connection or modify the mapping

# Check if ID formats are compatible by looking at a subset
common_ids = set(gene_data.index) & set(mapping_df['ID'])
print(f"\nNumber of common IDs between gene_data and mapping_df: {len(common_ids)}")

# If no common IDs, we need to reformulate our approach
# Let's try to create a mapping using the 'ID' column from gene_data
# and extracting gene symbols from the 'gene_assignment' column

# Modified approach: create a new mapping using extracted gene symbols
gene_data_mapped = apply_gene_mapping(gene_data, mapping_df)

# Preview the result
print("\nGene expression data after mapping:")
print(f"Shape: {gene_data_mapped.shape}")
print(f"First few gene symbols: {gene_data_mapped.index[:10]}")


Gene data index preview:
Index(['2824546_st', '2824549_st', '2824551_st', '2824554_st', '2827992_st'], dtype='object', name='ID')

Checking ID in gene_annotation:
0    TC01000001.hg.1
1    TC01000002.hg.1
2    TC01000003.hg.1
3    TC01000004.hg.1
4    TC01000005.hg.1
Name: ID, dtype: object

Checking probeset_id in gene_annotation:
0    TC01000001.hg.1
1    TC01000002.hg.1
2    TC01000003.hg.1
3    TC01000004.hg.1
4    TC01000005.hg.1
Name: probeset_id, dtype: object

Mapping dataframe sample:
                ID                                               Gene
0  TC01000001.hg.1  NR_046018 // DDX11L1 // DEAD/H (Asp-Glu-Ala-As...
1  TC01000002.hg.1  ENST00000408384 // MIR1302-11 // microRNA 1302...

First few probe IDs in gene expression data:
Index(['2824546_st', '2824549_st', '2824551_st', '2824554_st', '2827992_st'], dtype='object', name='ID')

First few probe IDs in mapping dataframe:
0    TC01000001.hg.1
1    TC01000002.hg.1
2    TC01000003.hg.1
3    TC01000004.hg.1
4    TC010000


Gene expression data after mapping:
Shape: (71528, 33)
First few gene symbols: Index(['A-', 'A-2', 'A-52', 'A-575C2', 'A-E', 'A-I', 'A-II', 'A-IV', 'A-V',
       'A0'],
      dtype='object', name='Gene')


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
# First, let's examine a few gene symbols before normalization to understand the issue
gene_data_mapped = apply_gene_mapping(gene_data, mapping_df)
print(f"Gene data after mapping, before normalization: {gene_data_mapped.shape}")
print(f"Sample of gene symbols before normalization:")
print(gene_data_mapped.index[:20])  # Display first 20 gene symbols

# Try normalizing the mapped gene data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data_mapped)
print(f"Original gene count after mapping: {len(gene_data_mapped)}")
print(f"Normalized gene count: {len(normalized_gene_data)}")

# If normalization resulted in too few genes, use the mapped data without normalization
if len(normalized_gene_data) < 1000:  # Arbitrary threshold - if less than 1000 genes remain
    print("Warning: Gene symbol normalization removed too many genes. Using mapped gene data without normalization.")
    normalized_gene_data = gene_data_mapped  # Use the mapped data without normalization

# Create directory for the gene data file if it doesn't exist
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)

# Save the gene data to a CSV file
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Gene data saved to {out_gene_data_file}")

# 2. Load clinical data from the CSV file we saved in a previous step
try:
    selected_clinical_df = pd.read_csv(out_clinical_data_file, index_col=0)
except FileNotFoundError:
    # If the file doesn't exist, extract from matrix file again
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    _, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
    
    # Extract clinical features with the correct sample IDs
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )

print(f"Clinical data shape: {selected_clinical_df.shape}")
print("First few sample IDs in clinical data:")
print(list(selected_clinical_df.columns)[:5])
print("First few sample IDs in gene data:")
print(list(normalized_gene_data.columns)[:5])

# Link the clinical and genetic data
linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
print(f"Linked data shape: {linked_data.shape}")

# Check if we have enough valid data after linking
if linked_data.shape[0] <= 1 or linked_data.shape[1] <= 1:
    print("Warning: Linked data has insufficient dimensions")
    is_usable = validate_and_save_cohort_info(
        is_final=True, 
        cohort=cohort, 
        info_path=json_path, 
        is_gene_available=True, 
        is_trait_available=True, 
        is_biased=True,  # Consider it biased if linking fails
        df=linked_data, 
        note="Data linking failed - insufficient dimensions in linked data."
    )
    print("The dataset was determined to be not usable for analysis.")
else:
    # 3. Handle missing values in the linked data
    linked_data_cleaned = handle_missing_values(linked_data, trait)
    
    print(f"Linked data shape after handling missing values: {linked_data_cleaned.shape}")
    
    # Check if we still have enough data after cleaning
    if linked_data_cleaned.shape[0] < 10 or linked_data_cleaned.shape[1] < 10:
        print("Warning: After handling missing values, insufficient data remains for analysis")
        is_usable = validate_and_save_cohort_info(
            is_final=True, 
            cohort=cohort, 
            info_path=json_path, 
            is_gene_available=True, 
            is_trait_available=True, 
            is_biased=True,
            df=linked_data_cleaned, 
            note="After cleaning, insufficient data remains for analysis."
        )
        print("The dataset was determined to be not usable for analysis.")
    else:
        # 4. Determine whether the trait and demographic features are severely biased
        is_trait_biased, linked_data_cleaned = judge_and_remove_biased_features(linked_data_cleaned, trait)
        
        # 5. Conduct quality check and save the cohort information.
        note = "Dataset contains gene expression data from bladder cancer cell lines."
        is_usable = validate_and_save_cohort_info(
            is_final=True, 
            cohort=cohort, 
            info_path=json_path, 
            is_gene_available=True, 
            is_trait_available=True, 
            is_biased=is_trait_biased, 
            df=linked_data_cleaned, 
            note=note
        )
        
        # 6. If the linked data is usable, save it as a CSV file.
        if is_usable:
            os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
            linked_data_cleaned.to_csv(out_data_file)
            print(f"Linked data saved to {out_data_file}")
        else:
            print(f"The dataset was determined to be not usable for analysis. Bias in trait: {is_trait_biased}")

Gene data after mapping, before normalization: (71528, 33)
Sample of gene symbols before normalization:
Index(['A-', 'A-2', 'A-52', 'A-575C2', 'A-E', 'A-I', 'A-II', 'A-IV', 'A-V',
       'A0', 'A1', 'A1-', 'A10', 'A11', 'A12', 'A13', 'A14', 'A16', 'A18',
       'A19'],
      dtype='object', name='Gene')
Original gene count after mapping: 71528
Normalized gene count: 24018


Gene data saved to ../../output/preprocess/Bladder_Cancer/gene_data/GSE201395.csv
Clinical data shape: (1, 33)
First few sample IDs in clinical data:
['GSM6062606', 'GSM6062607', 'GSM6062608', 'GSM6062609', 'GSM6062610']
First few sample IDs in gene data:
['GSM6062606', 'GSM6062607', 'GSM6062608', 'GSM6062609', 'GSM6062610']
Linked data shape: (33, 24019)


Linked data shape after handling missing values: (33, 24019)
Quartiles for 'Bladder_Cancer':
  25%: 1.0
  50% (Median): 1.0
  75%: 1.0
Min: 1.0
Max: 1.0
The distribution of the feature 'Bladder_Cancer' in this dataset is severely biased.

The dataset was determined to be not usable for analysis. Bias in trait: True
